In [8]:
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import pandas as pd
import os
import torch
from tqdm import tqdm

### Data Loading

Define data paths

In [ ]:
PATH = "" # Path to the source data
IMAGES_PATH = f"{PATH}/archive/images/images/"
CSV_PATH = f"{PATH}/archive/"
HEIGHT = 224
WIDTH = 160

Load **Fashion CLIP** from the **HuggingFACE API**

In [ ]:
model = CLIPModel.from_pretrained("patrickjohncyh/fashion-clip").to("cuda")
processor = CLIPProcessor.from_pretrained("patrickjohncyh/fashion-clip")

Load .csv and filter for efficient embedding computation

In [11]:
product_df = pd.read_csv(f"{CSV_PATH}product_data.csv")
test_df = pd.read_csv(f"{CSV_PATH}test_data.csv")
test_df = test_df.drop_duplicates(subset='cod_modelo_color')

Helper function for image loading

In [13]:
def load_imagePath_2_PIL(img_path):
    return Image.open(f"{IMAGES_PATH}/{img_path}")

### Embedding computation

Define the output file for the **train** dataset

In [ ]:
embeddings_file = "image_embeddings.pt"

Compute embeddings and store them as a dict (*img_path*: *img_embedding*)

In [ ]:
embeddings = {}
with torch.no_grad():  # Disable gradient calculation for inference
    for img_name in tqdm(product_df['des_filename']):
        img_path = f"{IMAGES_PATH}/{img_name}"
        try:
            image = Image.open(img_path)
        except:
            print(f"Error opening image {img_path}")

        # Preprocess the image with the FASHION-CLIP processor
        inputs = processor(images=image, return_tensors="pt", padding=True).to("cuda")
        
        # Get image embeddings
        image_features = model.get_image_features(**inputs)
        image_features = image_features.squeeze().cpu()

        # Store embeddings in a dictionary
        embeddings[img_name] = image_features

# Save embeddings to a file
torch.save(embeddings, embeddings_file)
print(f"Embeddings saved to {embeddings_file}")

Make sure that the embeddings are correctly stored and loaded

In [ ]:
embeddings_file = "image_embeddings.pt"
embeddings = torch.load(embeddings_file)

Repeat the process for the **test** dataset

In [ ]:
embeddings_file = "image_embeddings_test.pt"

embeddings_test = {}
with torch.no_grad():  # Disable gradient calculation for inference
    for img_name in tqdm(test_df['des_filename']):
        img_path = f"{IMAGES_PATH}/{img_name}"
        try:
            image = Image.open(img_path)
        except:
            print(f"Error opening image {img_path}")

        # Preprocess the image
        inputs = processor(images=image, return_tensors="pt", padding=True).to("cuda")
        
        # Get image embeddings
        image_features = model.get_image_features(**inputs)
        image_features = image_features.squeeze().cpu()  # Convert to a 1D tensor and move to CPU

        # Store embeddings in a dictionary
        embeddings_test[img_name] = image_features

torch.save(embeddings_test, embeddings_file)
print(f"Embeddings saved to {embeddings_file}")